In [34]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib as plt
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/talkingdata-adtracking-fraud-detection/sample_submission.csv
/kaggle/input/talkingdata-adtracking-fraud-detection/train_sample.csv
/kaggle/input/talkingdata-adtracking-fraud-detection/test_supplement.csv
/kaggle/input/talkingdata-adtracking-fraud-detection/train.csv
/kaggle/input/talkingdata-adtracking-fraud-detection/test.csv


# Preproccess Data 
- split data into training and labels
- change click times into integers
- convert data into floats 

In [28]:
# read in training data
training_data = pd.read_csv("/kaggle/input/talkingdata-adtracking-fraud-detection/train_sample.csv")

In [42]:
# read in testing data
testing_data = pd.read_csv('/kaggle/input/talkingdata-adtracking-fraud-detection/test_supplement.csv')

In [29]:
# drop columns that will not be used 
training_data = training_data.drop(['attributed_time'], axis=1)

# split training data and labels
training_labels = training_data['is_attributed']

# remove labels from training data
training_data = training_data.drop(['is_attributed'], axis=1)

In [30]:
# change click_time column to a combined string and then cast into an int
def create_combined_datetime(date_time):
    date_time = date_time.split()
    date = date_time[0]
    time = date_time[1]
    date_combined = "".join(date.split("-"))
    time_combined = "".join(time.split(":"))
    date_time_combined = "".join([date_combined, time_combined])
    return int(date_time_combined)

training_data['click_time']=training_data['click_time'].apply(create_combined_datetime)

In [31]:
training_data_as_float = training_data.astype(float)
training_data_as_float_array = training_data_as_float.to_numpy()

In [32]:
# define a callback class
class myCallback(tf.keras.callbacks.Callback):
        # Define the correct function signature for on_epoch_end
        def on_epoch_end(self, epoch, logs={}):
            if logs.get('accuracy') is not None and logs.get('accuracy') > 0.99: # @KEEP
                print("\nReached 99% accuracy so cancelling training!") 
                
                # Stop training once the above condition is met
                self.model.stop_training = True

In [37]:
training_data_as_float.shape

(100000, 6)

In [40]:
# train data
def train_talking_data(x_train, y_train):
    mycallback = myCallback()
    model = tf.keras.models.Sequential([ 
        keras.layers.Flatten(input_shape=(6,)),
        keras.layers.Dense(units=512),
        keras.layers.Dense(units=1)
    ]) 
    optimizer = RMSprop(learning_rate=0.01)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(x_train, y_train, epochs=50, callbacks=[mycallback])
    return history, model

hist, mer_model = train_talking_data(training_data_as_float_array, training_labels)

Epoch 1/50
3120/3125 [============================>.] - ETA: 0s - loss: 0.0349 - accuracy: 0.9977
Reached 99% accuracy so cancelling training!
3125/3125 [==============================] - 6s 2ms/step - loss: 0.0350 - accuracy: 0.9977
